# Exome Pipeline

In [2]:
import subprocess
import os
import sys
from shlex import split

from IPython.display import display, Markdown, Latex

import numpy as np 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import functools

from interactPlots import *

import yaml

In [3]:
#Read options file

with open("options.yaml", 'r') as stream:
    data = yaml.safe_load(stream)

In [4]:
#Set environment variables

os.environ["SENTIEON_INSTALL_DIR"] = data['SENTIEON_INSTALL_DIR']
os.environ["SENTIEON_LICENSE"]= data['SENTIEON_LICENSE']
os.environ["BCFTOOLS_PLUGINS"]= data['bcfdir']

In [5]:
#Set working directory

data['home'] = os.getcwd()+"/"
#data['workdir'] = "/home/adefalco/"+ "/" + data['batch'] + "/"
data['workdir'] = data['home'] + data['batch'] + "/"

if not os.path.isdir(data['workdir']): 
    os.mkdir(data['workdir']) 
 
data['SENTIEON_TMPDIR'] = data['workdir']+"/$SAMPLE/tmpdir"

## Batch

In [46]:
#Read batch
from IPython.display import HTML

data['samplelist'] = data['home']+data['batch']+".txt"

batch = []
with open (data['samplelist'], 'r') as f:
    for row in csv.reader(f,delimiter='\t'):
            batch.append(row)
           
batchSize = int(np.size(batch)/np.size(batch,1))        
HTML(tabulate(batch, headers =["Sample","Tumor", "Normal"], tablefmt="html"))

Sample,Tumor,Normal
ABT414_Flank,A1783,control
ABT414_Flank,A9155,control
ABT414_Flank,A9878,control
ABT414_Flank,AT2550,control
ABT414_Flank,AT2822,control
ABT414_Flank,AT4415,control
ABT414_Flank,AT4808,control
ABT414_Flank,C0288,control
ABT414_Flank,C0334,control
ABT414_Flank,C1572,control


In [7]:
%%capture
#Dropdown widgets

tumors = [row[1] for row in batch]

widgTum = widgets.Dropdown(
    options= tumors,
    disabled=False,
)  

lists = tumors.copy()
lists.append('ALL')

widgTumAll = widgets.Dropdown(
    options= lists,
    value='ALL',
    disabled=False,
) 

normals = [row[0] for row in batch]
#normals = [batch[0][0]]

uniqNorm = np.unique(normals)

#lists = normals.copy()
#lists.append('ALL')

widgNor = widgets.Dropdown(
    options= uniqNorm,
    #value='ALL',
    disabled=False,
)  


In [11]:
os.chdir(data['workdir'])
exec_scripts = False #Run shell scripts

# 1a. Mapping reads with BWA-MEM, sorting for normal sample
The results of this call are dependent on the number of threads used. To have number of threads independent results, add chunk size option -K 10000000

In [93]:
ERROR = "error"
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    
    command = "(" + data['SENTIEON_INSTALL_DIR']+"/bin/sentieon bwa mem -M -R '@RG\\tID:"+SAMPLE+"N\\tSM:"+SAMPLE+"N\\tPL:"+data['platform']+"' -t "+str(data['nt'])+" -K 10000000 "+data['fasta']+" "+data['fastq_folder_NORMAL']+"/*_"+data['fastq_1_suffix']+" "+data['fastq_folder_NORMAL']+"/*_"+data['fastq_2_suffix']+" || echo -n "+ ERROR +" ) | "+data['SENTIEON_INSTALL_DIR']+"/bin/sentieon util sort -o "+SAMPLE+"N_sorted.bam -t "+str(data['nt'])+" --sam2bam -i -"
    print(command+'\n')
    if exec_scripts:
        subprocess.check_output(command, shell=True)

### Normal: ABT414_Flank

(/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon bwa mem -M -R '@RG\tID:ABT414_FlankN\tSM:ABT414_FlankN\tPL:ILLUMINA' -t 80 -K 10000000 /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_G12_control/controlbam/XenomeData/*_human_1.fastq /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_G12_control/controlbam/XenomeData/*_human_2.fastq || echo -n error ) | /storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon util sort -o ABT414_FlankN_sorted.bam -t 80 --sam2bam -i -



# 1a. Mapping reads with BWA-MEM, sorting for tumor sample
The results of this call are dependent on the number of threads used. To have number of threads independent results, add chunk size option -K 10000000

In [91]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command = "(" + data['SENTIEON_INSTALL_DIR']+"/bin/sentieon bwa mem -M -R '@RG\\tID:"+TUMOR+"T\\tSM:"+TUMOR+"T\\tPL:"+data['platform']+"' -t "+str(data['nt'])+" -K 10000000 "+data['fasta']+" "+data['fastq_folder']+"/"+TUMOR+"/*_"+data['fastq_1_suffix']+" "+data['fastq_folder']+"/"+TUMOR+"/*_"+data['fastq_2_suffix']+" || echo -n "+ ERROR +" ) | "+data['SENTIEON_INSTALL_DIR']+"/bin/sentieon util sort -o "+TUMOR+"T_sorted.bam -t "+str(data['nt'])+" --sam2bam -i -"
    print(command+'\n')
    if exec_scripts:
        subprocess.check_output(command, shell=True)

### Tumor: A1783

(/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon bwa mem -M -R '@RG\tID:A1783T\tSM:A1783T\tPL:ILLUMINA' -t 80 -K 10000000 /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/A1783/*_human_1.fastq /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/A1783/*_human_2.fastq || echo -n error ) | /storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon util sort -o A1783T_sorted.bam -t 80 --sam2bam -i -



### Tumor: A9155

(/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon bwa mem -M -R '@RG\tID:A9155T\tSM:A9155T\tPL:ILLUMINA' -t 80 -K 10000000 /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/A9155/*_human_1.fastq /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/A9155/*_human_2.fastq || echo -n error ) | /storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon util sort -o A9155T_sorted.bam -t 80 --sam2bam -i -



### Tumor: A9878

...

(/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon bwa mem -M -R '@RG\tID:C0334T\tSM:C0334T\tPL:ILLUMINA' -t 80 -K 10000000 /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/C0334/*_human_1.fastq /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/C0334/*_human_2.fastq || echo -n error ) | /storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon util sort -o C0334T_sorted.bam -t 80 --sam2bam -i -



### Tumor: C1572

(/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon bwa mem -M -R '@RG\tID:C1572T\tSM:C1572T\tPL:ILLUMINA' -t 80 -K 10000000 /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/C1572/*_human_1.fastq /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/C1572/*_human_2.fastq || echo -n error ) | /storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon util sort -o C1572T_sorted.bam -t 80 --sam2bam -i -



### Tumor: C8448

(/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon bwa mem -M -R '@RG\tID:C8448T\tSM:C8448T\tPL:ILLUMINA' -t 80 -K 10000000 /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/C8448/*_human_1.fastq /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414/ABT414_Flank/WES/RawData/XenomeData/C8448/*_human_2.fastq || echo -n error ) | /storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon util sort -o C8448T_sorted.bam -t 80 --sam2bam -i -



# 2a. Metrics for normal sample

In [21]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+SAMPLE+"N_sorted.bam --algo MeanQualityByCycle "+SAMPLE+"N_mq_metrics.txt --algo QualDistribution "+SAMPLE+"N_qd_metrics.txt --algo GCBias --summary "+SAMPLE+"N_gc_summary.txt "+SAMPLE+"N_gc_metrics.txt --algo AlignmentStat --adapter_seq '' "+SAMPLE+"N_aln_metrics.txt --algo InsertSizeMetricAlgo "+SAMPLE+"N_is_metrics.txt"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)


/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i ABT414_FlankN_sorted.bam --algo MeanQualityByCycle ABT414_FlankN_mq_metrics.txt --algo QualDistribution ABT414_FlankN_qd_metrics.txt --algo GCBias --summary ABT414_FlankN_gc_summary.txt ABT414_FlankN_gc_metrics.txt --algo AlignmentStat --adapter_seq '' ABT414_FlankN_aln_metrics.txt --algo InsertSizeMetricAlgo ABT414_FlankN_is_metrics.txt



In [9]:
%%capture
"""from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import functools
from interactPlots import *

inter = interact(tableShow, Sample=widgNor, file=fixed("_N_aln_metrics.txt"), cols = fixed([0,1,2,5,6]), listSample = fixed([]));
display(inter)"""

In [36]:


javascript = """

<script type="text/Javascript">

function myTable(idd, jsonObj) {

  var x = document.getElementById("selectSample"+idd).value;
  document.getElementById("myText"+idd).innerHTML = x;
  
  var table = document.getElementById("myTable"+idd);

  while(table.rows[0]) table.deleteRow(0);

  i = 0;
  
  var jsonObj2 = jsonObj[x];
  
  console.log(jsonObj2)
  
  Object.keys(jsonObj2).forEach(function(k){
  
    var row = table.insertRow(i);
    i = i+1;
    var cell1 = row.insertCell(0);
    var cell2 = row.insertCell(1);
    var cell3 = row.insertCell(2);
    var cell4 = row.insertCell(3);
    cell1.innerHTML = k;
    cell2.innerHTML = jsonObj2[k].TOTAL_READS;
    cell3.innerHTML = jsonObj2[k].PF_READS_ALIGNED;
    cell4.innerHTML = jsonObj2[k].PCT_PF_READS_ALIGNED;
    
  }); 
  
     var header = table.createTHead();
    var row = header.insertRow(0);     
    var cell1 = row.insertCell(0);
    var cell2 = row.insertCell(1);
    var cell3 = row.insertCell(2);
    var cell4 = row.insertCell(3);
    cell1.innerHTML = "CATEGORY";
    cell2.innerHTML = "TOTAL_READS";
    cell3.innerHTML = "PF_READS_ALIGNED";
    cell4.innerHTML = "PCT_PF_READS_ALIGNED";
}

function myDropdown(sel,opts) {
    for(var i = 0; i < opts.length; i++) {
        var opt = opts[i];
        var el = document.createElement("option");
        el.textContent = opt;
        el.value = opt;
        if (i < 1) {
            el.selected = "true";
        }
        sel.appendChild(el);
    }
}

function myPlots(type,idd) {
  var x = document.getElementById("selectSample"+idd).value;
  document.getElementById("myText"+idd).innerHTML = x;
  document.getElementById("pdf"+idd).src = x+type;
}

</script>

"""

HTML(javascript)

In [37]:
path = "/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/"
Sample = normals[0] #"ABT414_Flank"
file = "_N_aln_metrics.txt"

alldata = {}

for i in range(np.size(uniqNorm)):
    csvFilePath = path+uniqNorm[i]+file
    jsonfile = make_json(csvFilePath,uniqNorm[i],alldata)

#print(jsonfile)

id = 0
id = id +1

javascript = """


<head>

<style>
table, td {
  border: 1px solid black;
  font-size: 18px;
}
</style>

</head>


<select id='selectSample"""+str(id)+"""'>  </select>

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<body>
<table id="myTable"""+str(id)+""""></table>
<br>

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

var objNorm = JSON.parse('"""+jsonfile+"""');

myDropdown(select,Object.keys(objNorm)) 

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myTable("""+str(id)+""",objNorm)};

myTable("""+str(id)+""",objNorm)

</script>
</body>
"""

HTML(javascript)

- **CATEGORY**: Distinguishes either 
    - PAIRED: for a fragment run.
    - FIRST_OF_PAIR: when metrics are for only the first read in a paired run. 
    - SECOND_OF_PAIR: when metrics are for only the second read in a paired run.
    - PAIR: when metrics are aggregated for both first and second reads in a pair. 


- **TOTAL_READS**: The total number of reads 

- **PF_READS_ALIGNED**: The number of reads that aligned to the reference sequence

- **PCT_PF_READS_ALIGNED**: The percentage of reads that aligned to the reference sequence

In [31]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot GCBias -o "+SAMPLE+"N_gc-report.pdf "+SAMPLE+"N_gc_metrics.txt"
    print(command2+'\n')
    if exec_scripts:        
        subprocess.check_output(command2,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot GCBias -o ABT414_FlankN_gc-report.pdf ABT414_FlankN_gc_metrics.txt



NameError: name 'exec_scripts' is not defined

In [1]:
#interact(plots, Sample=widgNor, file=fixed("_N_gc-report.pdf"), normal=fixed(True), listSample = fixed([]));
#embed_minimal_html('export.html', views=test, title='Widgets export')



id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objNorm)) 

var type"""+str(id)+""" = "_N_gc-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};

</script>
"""

HTML(input_form + javascript)


TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'int'

In [95]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command3 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot QualDistribution -o "+SAMPLE+"N_qd-report.pdf "+SAMPLE+"N_qd_metrics.txt"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualDistribution -o ABT414_FlankN_qd-report.pdf ABT414_FlankN_qd_metrics.txt



In [92]:
#interact(plots, Sample=widgNor, file=fixed("_N_qd-report.pdf"), normal=fixed(True), listSample = fixed([]));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objNorm)) 

var type"""+str(id)+""" = "_N_qd-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)



In [97]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command4 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot MeanQualityByCycle -o "+SAMPLE+"N_mq-report.pdf "+SAMPLE+"N_mq_metrics.txt"
    print(command4+'\n')
    if exec_scripts:
        subprocess.check_output(command4,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot MeanQualityByCycle -o ABT414_FlankN_mq-report.pdf ABT414_FlankN_mq_metrics.txt



In [94]:
#interact(plots, Sample=widgNor, file=fixed("_N_mq-report.pdf"), normal=fixed(True), listSample = fixed([]));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objNorm)) 

var type"""+str(id)+""" = "_N_mq-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)



In [98]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command5 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot InsertSizeMetricAlgo -o "+SAMPLE+"N_is-report.pdf "+SAMPLE+"N_is_metrics.txt"
    print(command5+'\n')
    if exec_scripts:
        subprocess.check_output(command5,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot InsertSizeMetricAlgo -o ABT414_FlankN_is-report.pdf ABT414_FlankN_is_metrics.txt



In [96]:
#interact(plots, Sample=widgNor, file=fixed("_N_is-report.pdf"), normal=fixed(True), listSample = fixed([]));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objNorm)) 

var type"""+str(id)+""" = "_N_is-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)



# 2a. Metrics for tumor sample

In [87]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_sorted.bam --algo MeanQualityByCycle "+TUMOR+"T_mq_metrics.txt --algo QualDistribution "+TUMOR+"T_qd_metrics.txt --algo GCBias --summary "+TUMOR+"T_gc_summary.txt "+TUMOR+"T_gc_metrics.txt --algo AlignmentStat --adapter_seq '' "+TUMOR+"T_aln_metrics.txt --algo InsertSizeMetricAlgo "+TUMOR+"T_is_metrics.txt"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_sorted.bam --algo MeanQualityByCycle A1783T_mq_metrics.txt --algo QualDistribution A1783T_qd_metrics.txt --algo GCBias --summary A1783T_gc_summary.txt A1783T_gc_metrics.txt --algo AlignmentStat --adapter_seq '' A1783T_aln_metrics.txt --algo InsertSizeMetricAlgo A1783T_is_metrics.txt



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_sorted.bam --algo MeanQualityByCycle A9155T_mq_metrics.txt --algo QualDistribution A9155T_qd_metrics.txt --algo GCBias --summary A9155T_gc_summary.txt A9155T_gc_metrics.txt --algo AlignmentStat --adapter_seq '' A9155T_aln_metrics.txt --algo InsertSizeMetricAlgo A9155T_is_metrics.txt



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_sorted.bam --algo MeanQualityByCycle C0334T_mq_metrics.txt --algo QualDistribution C0334T_qd_metrics.txt --algo GCBias --summary C0334T_gc_summary.txt C0334T_gc_metrics.txt --algo AlignmentStat --adapter_seq '' C0334T_aln_metrics.txt --algo InsertSizeMetricAlgo C0334T_is_metrics.txt



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_sorted.bam --algo MeanQualityByCycle C1572T_mq_metrics.txt --algo QualDistribution C1572T_qd_metrics.txt --algo GCBias --summary C1572T_gc_summary.txt C1572T_gc_metrics.txt --algo AlignmentStat --adapter_seq '' C1572T_aln_metrics.txt --algo InsertSizeMetricAlgo C1572T_is_metrics.txt



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_sorted.bam --algo MeanQualityByCycle C8448T_mq_metrics.txt --algo QualDistribution C8448T_qd_metrics.txt --algo GCBias --summary C8448T_gc_summary.txt C8448T_gc_metrics.txt --algo AlignmentStat --adapter_seq '' C8448T_aln_metrics.txt --algo InsertSizeMetricAlgo C8448T_is_metrics.txt



CATEGORY: Distinguishes either PAIRED: for a fragment run, FIRST_OF_PAIR: when metrics are for only the first read in a paired run, SECOND_OF_PAIR: when metrics are for only the second read in a paired run, or PAIR when metrics are aggregated for both first and second reads in a pair. 

Total Reads: The total number of reads 

PF_READS_ALIGNED: The number of reads that aligned to the reference sequence

PCT_PF_READS_ALIGNED: The percentage of reads that aligned to the reference sequence

In [98]:
%%capture
"""interact(tableShow, Sample=widgTumAll, file=fixed("_T_aln_metrics.txt"), cols = fixed([0,1,2,5,6]), listSample = fixed(tumors));"""

'interact(tableShow, Sample=widgTumAll, file=fixed("_T_aln_metrics.txt"), cols = fixed([0,1,2,5,6]), listSample = fixed(tumors));'

In [38]:
path = "/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/"
#Sample = tumors[1]
file = "_T_aln_metrics.txt"    

alldata = {}

for i in range(batchSize):
    csvFilePath = path+tumors[i]+file
    jsonfile = make_json(csvFilePath,tumors[i],alldata)
    
id = id +1

javascript = """

<select id='selectSample"""+str(id)+"""'>  </select>

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<body>

<table id="myTable"""+str(id)+"""""></table>
<br>

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

var objTum = JSON.parse('"""+jsonfile+"""');

myDropdown(select,Object.keys(objTum)) 

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myTable("""+str(id)+""",objTum)};

myTable("""+str(id)+""",objTum)

</script>
</body>
"""

HTML(javascript)

- **CATEGORY**: Distinguishes either 
    - PAIRED: for a fragment run.
    - FIRST_OF_PAIR: when metrics are for only the first read in a paired run. 
    - SECOND_OF_PAIR: when metrics are for only the second read in a paired run.
    - PAIR: when metrics are aggregated for both first and second reads in a pair. 


- **TOTAL_READS**: The total number of reads 

- **PF_READS_ALIGNED**: The number of reads that aligned to the reference sequence

- **PCT_PF_READS_ALIGNED**: The percentage of reads that aligned to the reference sequence

In [99]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot GCBias -o "+TUMOR+"T_gc-report.pdf "+TUMOR+"T_gc_metrics.txt"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot GCBias -o A1783T_gc-report.pdf A1783T_gc_metrics.txt



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot GCBias -o A9155T_gc-report.pdf A9155T_gc_metrics.txt



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot GCBias -o C0334T_gc-report.pdf C0334T_gc_metrics.txt



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot GCBias -o C1572T_gc-report.pdf C1572T_gc_metrics.txt



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot GCBias -o C8448T_gc-report.pdf C8448T_gc_metrics.txt



In [101]:
#interact(plots, Sample=widgTumAll, file=fixed("_T_gc-report.pdf"), normal=fixed(False), listSample = fixed(tumors));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objTum)) 

var type"""+str(id)+""" = "_T_gc-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)

In [100]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command3 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot QualDistribution -o "+TUMOR+"T_qd-report.pdf "+TUMOR+"T_qd_metrics.txt"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualDistribution -o A1783T_qd-report.pdf A1783T_qd_metrics.txt



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualDistribution -o A9155T_qd-report.pdf A9155T_qd_metrics.txt



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualDistribution -o C0334T_qd-report.pdf C0334T_qd_metrics.txt



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualDistribution -o C1572T_qd-report.pdf C1572T_qd_metrics.txt



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualDistribution -o C8448T_qd-report.pdf C8448T_qd_metrics.txt



In [103]:
#interact(plots, Sample=widgTumAll, file=fixed("_T_qd-report.pdf"), normal=fixed(False), listSample = fixed(tumors));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objTum)) 

var type"""+str(id)+""" = "_T_qd-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)

In [101]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command4 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot MeanQualityByCycle -o "+TUMOR+"T_mq-report.pdf "+TUMOR+"T_mq_metrics.txt"
    print(command4+'\n')
    if exec_scripts:
        subprocess.check_output(command4,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot MeanQualityByCycle -o A1783T_mq-report.pdf A1783T_mq_metrics.txt



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot MeanQualityByCycle -o A9155T_mq-report.pdf A9155T_mq_metrics.txt



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot MeanQualityByCycle -o C0334T_mq-report.pdf C0334T_mq_metrics.txt



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot MeanQualityByCycle -o C1572T_mq-report.pdf C1572T_mq_metrics.txt



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot MeanQualityByCycle -o C8448T_mq-report.pdf C8448T_mq_metrics.txt



In [105]:
#interact(plots, Sample=widgTumAll, file=fixed("_T_mq-report.pdf"), normal=fixed(False), listSample = fixed(tumors));
#embed_minimal_html('export.html', views=[widgTumAll], title='Widgets export')

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objTum)) 

var type"""+str(id)+""" = "_T_mq-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)

In [102]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command5 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot InsertSizeMetricAlgo -o "+TUMOR+"T_is-report.pdf "+TUMOR+"T_is_metrics.txt"
    print(command5+'\n')
    if exec_scripts:
        subprocess.check_output(command5,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot InsertSizeMetricAlgo -o A1783T_is-report.pdf A1783T_is_metrics.txt



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot InsertSizeMetricAlgo -o A9155T_is-report.pdf A9155T_is_metrics.txt



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot InsertSizeMetricAlgo -o C0334T_is-report.pdf C0334T_is_metrics.txt



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot InsertSizeMetricAlgo -o C1572T_is-report.pdf C1572T_is_metrics.txt



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot InsertSizeMetricAlgo -o C8448T_is-report.pdf C8448T_is_metrics.txt



In [107]:
#interact(plots, Sample=widgTumAll, file=fixed("_T_is-report.pdf"), normal=fixed(False), listSample = fixed(tumors));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objTum)) 

var type"""+str(id)+""" = "_T_is-report.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)

# 3a. Remove Duplicate Reads for normal sample

In [84]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -i "+SAMPLE+"N_sorted.bam --algo LocusCollector --fun score_info "+SAMPLE+"N_score.txt"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i ABT414_FlankN_sorted.bam --algo LocusCollector --fun score_info ABT414_FlankN_score.txt



In [103]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -i "+SAMPLE+"N_sorted.bam --algo Dedup --rmdup --score_info "+SAMPLE+"N_score.txt --metrics "+SAMPLE+"N_dedup_metrics.txt "+SAMPLE+"N_deduped.bam"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i ABT414_FlankN_sorted.bam --algo Dedup --rmdup --score_info ABT414_FlankN_score.txt --metrics ABT414_FlankN_dedup_metrics.txt ABT414_FlankN_deduped.bam



# 3a. Remove Duplicate Reads for tumor sample

In [82]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -i "+TUMOR+"T_sorted.bam --algo LocusCollector --fun score_info "+TUMOR+"T_score.txt"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i A1783T_sorted.bam --algo LocusCollector --fun score_info A1783T_score.txt



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i A9155T_sorted.bam --algo LocusCollector --fun score_info A9155T_score.txt



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i C0334T_sorted.bam --algo LocusCollector --fun score_info C0334T_score.txt



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i C1572T_sorted.bam --algo LocusCollector --fun score_info C1572T_score.txt



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i C8448T_sorted.bam --algo LocusCollector --fun score_info C8448T_score.txt



In [104]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -i "+TUMOR+"T_sorted.bam --algo Dedup --rmdup --score_info "+TUMOR+"T_score.txt --metrics "+TUMOR+"T_dedup_metrics.txt "+TUMOR+"T_deduped.bam"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i A1783T_sorted.bam --algo Dedup --rmdup --score_info A1783T_score.txt --metrics A1783T_dedup_metrics.txt A1783T_deduped.bam



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i A9155T_sorted.bam --algo Dedup --rmdup --score_info A9155T_score.txt --metrics A9155T_dedup_metrics.txt A9155T_deduped.bam



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i C0334T_sorted.bam --algo Dedup --rmdup --score_info C0334T_score.txt --metrics C0334T_dedup_metrics.txt C0334T_deduped.bam



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i C1572T_sorted.bam --algo Dedup --rmdup --score_info C1572T_score.txt --metrics C1572T_dedup_metrics.txt C1572T_deduped.bam



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -i C8448T_sorted.bam --algo Dedup --rmdup --score_info C8448T_score.txt --metrics C8448T_dedup_metrics.txt C8448T_deduped.bam



# 4a. Indel realigner for normal sample

In [80]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+ "/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+SAMPLE+"N_deduped.bam --algo Realigner -k "+data['known_Mills_indels']+" -k "+data['known_1000G_indels']+" "+SAMPLE+"N_realigned.bam"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i ABT414_FlankN_deduped.bam --algo Realigner -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz ABT414_FlankN_realigned.bam



# 4a. Indel realigner for tumor sample

In [81]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = data['SENTIEON_INSTALL_DIR']+ "/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_deduped.bam --algo Realigner -k "+data['known_Mills_indels']+" -k "+data['known_1000G_indels']+" "+TUMOR+"T_realigned.bam"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_deduped.bam --algo Realigner -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz A1783T_realigned.bam



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_deduped.bam --algo Realigner -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz A9155T_realigned.bam



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_deduped.bam --algo Realigner -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C0334T_realigned.bam



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_deduped.bam --algo Realigner -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C1572T_realigned.bam



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_deduped.bam --algo Realigner -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C8448T_realigned.bam



# 5a. Base recalibration for normal sample

In [79]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+SAMPLE+"N_realigned.bam --algo QualCal -k "+data['dbsnp']+" -k "+data['known_Mills_indels']+" -k "+data['known_1000G_indels']+" "+SAMPLE+"N_recal_data.table"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i ABT414_FlankN_realigned.bam --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz ABT414_FlankN_recal_data.table



In [78]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+SAMPLE+"N_realigned.bam -q "+SAMPLE+"N_recal_data.table --algo QualCal -k "+data['dbsnp']+" -k "+data['known_Mills_indels']+" -k "+data['known_1000G_indels']+" "+SAMPLE+"N_recal_data.table.post"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i ABT414_FlankN_realigned.bam -q ABT414_FlankN_recal_data.table --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz ABT414_FlankN_recal_data.table.post



In [77]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command3 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" --algo QualCal --plot --before "+SAMPLE+"N_recal_data.table --after "+SAMPLE+"N_recal_data.table.post "+SAMPLE+"N_recal.csv"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 --algo QualCal --plot --before ABT414_FlankN_recal_data.table --after ABT414_FlankN_recal_data.table.post ABT414_FlankN_recal.csv



In [76]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command4 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot QualCal -o "+SAMPLE+"N_recal_plots.pdf "+SAMPLE+"N_recal.csv"
    print(command4+'\n')
    if exec_scripts:
        subprocess.check_output(command4,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualCal -o ABT414_FlankN_recal_plots.pdf ABT414_FlankN_recal.csv



In [118]:
#SAMPLE = batch[0][0]
#img = WImage(filename=SAMPLE+"N_recal_plots.pdf")
    
#interact(multiPage, Sample=widgNor, page=[1,2] , file=fixed("_N_recal_plots.pdf"), normal=fixed(True), listSample = fixed([]));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objNorm)) 

var type"""+str(id)+""" = "_N_recal_plots.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    

HTML(input_form + javascript)




In [75]:
display(Markdown("___"))
# ReadWriter to output recalibrated bam
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command5 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+SAMPLE+"N_realigned.bam -q "+SAMPLE+"N_recal_data.table --algo ReadWriter "+SAMPLE+"N_recal.bam"
    print(command5+'\n')
    if exec_scripts:
        subprocess.check_output(command5,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i ABT414_FlankN_realigned.bam -q ABT414_FlankN_recal_data.table --algo ReadWriter ABT414_FlankN_recal.bam



# 5a. Base recalibration for tumor sample

In [74]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_realigned.bam --algo QualCal -k "+data['dbsnp']+" -k "+data['known_Mills_indels']+" -k "+data['known_1000G_indels']+" "+TUMOR+"T_recal_data.table"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_realigned.bam --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz A1783T_recal_data.table



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_realigned.bam --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz A9155T_recal_data.table



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_realigned.bam --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C0334T_recal_data.table



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_realigned.bam --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C1572T_recal_data.table



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_realigned.bam --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C8448T_recal_data.table



In [73]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_realigned.bam -q "+TUMOR+"T_recal_data.table --algo QualCal -k "+data['dbsnp']+" -k "+data['known_Mills_indels']+" -k "+data['known_1000G_indels']+" "+TUMOR+"T_recal_data.table.post"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_realigned.bam -q A1783T_recal_data.table --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz A1783T_recal_data.table.post



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_realigned.bam -q A9155T_recal_data.table --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz A9155T_recal_data.table.post



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_realigned.bam -q C0334T_recal_data.table --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C0334T_recal_data.table.post



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_realigned.bam -q C1572T_recal_data.table --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C1572T_recal_data.table.post



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_realigned.bam -q C8448T_recal_data.table --algo QualCal -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/Mills_and_1000G_gold_standard.indels.vcf.gz -k /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/1000G_phase1.snps.high_confidence.vcf.gz C8448T_recal_data.table.post



In [72]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command3 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" --algo QualCal --plot --before "+TUMOR+"T_recal_data.table --after "+TUMOR+"T_recal_data.table.post "+TUMOR+"T_recal.csv"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 --algo QualCal --plot --before A1783T_recal_data.table --after A1783T_recal_data.table.post A1783T_recal.csv



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 --algo QualCal --plot --before A9155T_recal_data.table --after A9155T_recal_data.table.post A9155T_recal.csv



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 --algo QualCal --plot --before C0334T_recal_data.table --after C0334T_recal_data.table.post C0334T_recal.csv



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 --algo QualCal --plot --before C1572T_recal_data.table --after C1572T_recal_data.table.post C1572T_recal.csv



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 --algo QualCal --plot --before C8448T_recal_data.table --after C8448T_recal_data.table.post C8448T_recal.csv



In [71]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command4 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon plot QualCal -o "+TUMOR+"T_recal_plots.pdf "+TUMOR+"T_recal.csv"
    print(command4+'\n')
    if exec_scripts:
        subprocess.check_output(command4,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualCal -o A1783T_recal_plots.pdf A1783T_recal.csv



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualCal -o A9155T_recal_plots.pdf A9155T_recal.csv



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualCal -o C0334T_recal_plots.pdf C0334T_recal.csv



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualCal -o C1572T_recal_plots.pdf C1572T_recal.csv



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon plot QualCal -o C8448T_recal_plots.pdf C8448T_recal.csv



In [30]:
#interact(multiPage, Sample=widgTum, page=[1,2] , file=fixed("_T_recal_plots.pdf"), normal=fixed(True), listSample = fixed(tumors));

id = id +1

input_form = """

<select id='selectSample"""+str(id)+"""'>  </select>

<body onload="myPlots()">

<h1>The sample is:  <span id="myText"""+str(id)+""""></span></h1>

<div>
  <iframe id="pdf"""+str(id)+"""" src="" style="width:100%;height:700px;"></iframe>
</div>


</body>

"""

javascript = """

<script type="text/Javascript">

var select = document.getElementById("selectSample"""+str(id)+"""");

myDropdown(select,Object.keys(objTum)) 

var type"""+str(id)+""" = "_T_recal_plots.pdf"

myPlots(type"""+str(id)+""","""+str(id)+""")

document.getElementById("selectSample"""+str(id)+"""").onchange = function() {myPlots(type"""+str(id)+""","""+str(id)+""")};


</script>
"""
    
HTML(input_form + javascript)

In [105]:
display(Markdown("___"))
# ReadWriter to output recalibrated bam
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command5 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_realigned.bam -q "+TUMOR+"T_recal_data.table --algo ReadWriter "+TUMOR+"T_recal.bam"
    print(command5+'\n')
    if exec_scripts:
        subprocess.check_output(command5,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_realigned.bam -q A1783T_recal_data.table --algo ReadWriter A1783T_recal.bam



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_realigned.bam -q A9155T_recal_data.table --algo ReadWriter A9155T_recal.bam



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_realigned.bam -q C0334T_recal_data.table --algo ReadWriter C0334T_recal.bam



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_realigned.bam -q C1572T_recal_data.table --algo ReadWriter C1572T_recal.bam



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_realigned.bam -q C8448T_recal_data.table --algo ReadWriter C8448T_recal.bam



# 7a. HC Variant caller (normal)

In [69]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+SAMPLE+"N_recal.bam --algo Haplotyper -d "+data['dbsnp']+" --emit_conf=30 --call_conf=30 "+SAMPLE+"N-output-hc.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
            subprocess.check_output(command1,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i ABT414_FlankN_recal.bam --algo Haplotyper -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --emit_conf=30 --call_conf=30 ABT414_FlankN-output-hc.vcf.gz



# 7a. HC Variant caller (tumor)

In [68]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_recal.bam --algo Haplotyper -d "+data['dbsnp']+" --emit_conf=30 --call_conf=30 "+TUMOR+"T-output-hc.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_recal.bam --algo Haplotyper -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --emit_conf=30 --call_conf=30 A1783T-output-hc.vcf.gz



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_recal.bam --algo Haplotyper -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --emit_conf=30 --call_conf=30 A9155T-output-hc.vcf.gz



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_recal.bam --algo Haplotyper -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --emit_conf=30 --call_conf=30 C0334T-output-hc.vcf.gz



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_recal.bam --algo Haplotyper -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --emit_conf=30 --call_conf=30 C1572T-output-hc.vcf.gz



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_recal.bam --algo Haplotyper -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --emit_conf=30 --call_conf=30 C8448T-output-hc.vcf.gz



# 8a. Variant calling DNAscope (normal)

In [65]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -r "+data['fasta']+" -i "+SAMPLE+"N_recal.bam --algo DNAscope -d "+data['dbsnp']+" --model "+data['ML_MODEL_N']+" "+SAMPLE+"N-tmpDNAscope.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -i ABT414_FlankN_recal.bam --algo DNAscope -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model ABT414_FlankN-tmpDNAscope.vcf.gz



In [106]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -r "+data['fasta']+" --algo DNAModelApply --model "+data['ML_MODEL_N']+" -v "+SAMPLE+"N-tmpDNAscope.vcf.gz "+SAMPLE+"N-DNAscope.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo DNAModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model -v ABT414_FlankN-tmpDNAscope.vcf.gz ABT414_FlankN-DNAscope.vcf.gz



In [107]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command3 = data['bcfdir'] + " filter -s ML_FAIL -i INFO/ML_PROB > 0.81 "+SAMPLE+"N-DNAscope.vcf.gz -O z -m x -o "+SAMPLE+"N-filtDNAscope.vcf.gz"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i INFO/ML_PROB > 0.81 ABT414_FlankN-DNAscope.vcf.gz -O z -m x -o ABT414_FlankN-filtDNAscope.vcf.gz



# 8a. Variant calling DNAscope (tumor)

In [64]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -r "+data['fasta']+" -i "+TUMOR+"T_recal.bam --algo DNAscope -d "+data['dbsnp']+" --model "+data['ML_MODEL_N']+" "+TUMOR+"T-tmpDNAscope.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -i A1783T_recal.bam --algo DNAscope -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model A1783T-tmpDNAscope.vcf.gz



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -i A9155T_recal.bam --algo DNAscope -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model A9155T-tmpDNAscope.vcf.gz



### Tumor: A9878

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -i C0334T_recal.bam --algo DNAscope -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model C0334T-tmpDNAscope.vcf.gz



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -i C1572T_recal.bam --algo DNAscope -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model C1572T-tmpDNAscope.vcf.gz



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -i C8448T_recal.bam --algo DNAscope -d /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model C8448T-tmpDNAscope.vcf.gz



In [108]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -r "+data['fasta']+" --algo DNAModelApply --model "+data['ML_MODEL_N']+" -v "+TUMOR+"T-tmpDNAscope.vcf.gz "+TUMOR+"T-DNAscope.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo DNAModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model -v A1783T-tmpDNAscope.vcf.gz A1783T-DNAscope.vcf.gz



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo DNAModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model -v A9155T-tmpDNAscope.vcf.gz A9155T-DNAscope.vcf.gz



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo DNAModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model -v C0334T-tmpDNAscope.vcf.gz C0334T-DNAscope.vcf.gz



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo DNAModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model -v C1572T-tmpDNAscope.vcf.gz C1572T-DNAscope.vcf.gz



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo DNAModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonDNAscopeModelBeta0.4a-201808.05.model -v C8448T-tmpDNAscope.vcf.gz C8448T-DNAscope.vcf.gz



In [109]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command3 = data['bcfdir'] + " filter -s ML_FAIL -i INFO/ML_PROB > 0.81 "+TUMOR+"T-DNAscope.vcf.gz -O z -m x -o "+TUMOR+"T-filtDNAscope.vcf.gz"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i INFO/ML_PROB > 0.81 A1783T-DNAscope.vcf.gz -O z -m x -o A1783T-filtDNAscope.vcf.gz



### Tumor: A9155

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i INFO/ML_PROB > 0.81 A9155T-DNAscope.vcf.gz -O z -m x -o A9155T-filtDNAscope.vcf.gz



...

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i INFO/ML_PROB > 0.81 C0334T-DNAscope.vcf.gz -O z -m x -o C0334T-filtDNAscope.vcf.gz



### Tumor: C1572

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i INFO/ML_PROB > 0.81 C1572T-DNAscope.vcf.gz -O z -m x -o C1572T-filtDNAscope.vcf.gz



### Tumor: C8448

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i INFO/ML_PROB > 0.81 C8448T-DNAscope.vcf.gz -O z -m x -o C8448T-filtDNAscope.vcf.gz



# 9a. Variant Annotation (normal)

In [61]:
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command1 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+SAMPLE+"N-output-hc.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+SAMPLE+"N-output-hc.snpEff.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 ABT414_FlankN-output-hc.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > ABT414_FlankN-output-hc.snpEff.vcf.gz



In [110]:
display(Markdown("___"))
for i in range(np.size(uniqNorm)):
    SAMPLE = uniqNorm[i]
    display(Markdown("### Normal: " + SAMPLE))
    command2 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+SAMPLE+"N-filtDNAscope.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+SAMPLE+"N-filtDNAscope.snpEff.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 ABT414_FlankN-filtDNAscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > ABT414_FlankN-filtDNAscope.snpEff.vcf.gz



# 9a. Variant Annotation (tumor)

In [59]:
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command1 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+TUMOR+"T-output-hc.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+TUMOR+"T-output-hc.snpEff.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A1783T-output-hc.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A1783T-output-hc.snpEff.vcf.gz



### Tumor: A9155

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A9155T-output-hc.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A9155T-output-hc.snpEff.vcf.gz



### Tumor: A9878

...

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C0334T-output-hc.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C0334T-output-hc.snpEff.vcf.gz



### Tumor: C1572

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C1572T-output-hc.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C1572T-output-hc.snpEff.vcf.gz



### Tumor: C8448

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C8448T-output-hc.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C8448T-output-hc.snpEff.vcf.gz



In [111]:
display(Markdown("___"))
for i in range(batchSize):
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR))
    command2 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+TUMOR+"T-filtDNAscope.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+TUMOR+"T-filtDNAscope.snpEff.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A1783T-filtDNAscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A1783T-filtDNAscope.snpEff.vcf.gz



### Tumor: A9155

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A9155T-filtDNAscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A9155T-filtDNAscope.snpEff.vcf.gz



...

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C0334T-filtDNAscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C0334T-filtDNAscope.snpEff.vcf.gz



### Tumor: C1572

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C1572T-filtDNAscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C1572T-filtDNAscope.snpEff.vcf.gz



### Tumor: C8448

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C8448T-filtDNAscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C8448T-filtDNAscope.snpEff.vcf.gz



# 7b. Somatic Variant Calling TNseq

In [57]:
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_recal.bam -i "+SAMPLE+"N_recal.bam --algo TNsnv --tumor_sample "+TUMOR+"T --normal_sample "+SAMPLE+"N --pon "+data['panel_of_normal_TNsnv'] +" --cosmic "+data['cosmic_db']+" --dbsnp "+data['dbsnp']+" --call_stats_out "+TUMOR+"-call.stats "+TUMOR+"-TNsnv.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_recal.bam -i ABT414_FlankN_recal.bam --algo TNsnv --tumor_sample A1783T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --call_stats_out A1783-call.stats A1783-TNsnv.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_recal.bam -i ABT414_FlankN_recal.bam --algo TNsnv --tumor_sample A9155T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --call_stats_out A9155-call.stats A9155-TNsnv.vcf.gz



### Tumor: A9878 Normal: ABT414_Flank

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_recal.bam -i ABT414_FlankN_recal.bam --algo TNsnv --tumor_sample C0334T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --call_stats_out C0334-call.stats C0334-TNsnv.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_recal.bam -i ABT414_FlankN_recal.bam --algo TNsnv --tumor_sample C1572T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --call_stats_out C1572-call.stats C1572-TNsnv.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_recal.bam -i ABT414_FlankN_recal.bam --algo TNsnv --tumor_sample C8448T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --call_stats_out C8448-call.stats C8448-TNsnv.vcf.gz



In [112]:
display(Markdown("___"))
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_recal.bam -i "+SAMPLE+"N_recal.bam --algo TNhaplotyper --tumor_sample "+TUMOR+"T --normal_sample "+SAMPLE+"N --pon "+data['panel_of_normal_TNhaplotyper']+" --cosmic "+data['cosmic_db']+" --dbsnp "+data['dbsnp']+" "+TUMOR+"-TNhaplotyper.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_recal.bam -i ABT414_FlankN_recal.bam --algo TNhaplotyper --tumor_sample A1783T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz A1783-TNhaplotyper.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_recal.bam -i ABT414_FlankN_recal.bam --algo TNhaplotyper --tumor_sample A9155T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz A9155-TNhaplotyper.vcf.gz



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_recal.bam -i ABT414_FlankN_recal.bam --algo TNhaplotyper --tumor_sample C0334T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz C0334-TNhaplotyper.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_recal.bam -i ABT414_FlankN_recal.bam --algo TNhaplotyper --tumor_sample C1572T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz C1572-TNhaplotyper.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_recal.bam -i ABT414_FlankN_recal.bam --algo TNhaplotyper --tumor_sample C8448T --normal_sample ABT414_FlankN --pon /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/reference_PoNs_refseq_exome_10bp_hg19_300_1kg_normal_panel_CHR.vcf.gz --cosmic /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/cosmic.vcf.gz --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz C8448-TNhaplotyper.vcf.gz



# 8b. Somatic Variant calling TNscope

In [50]:
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command1 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -r "+data['fasta']+" -t "+str(data['nt'])+" -i "+TUMOR+"T_recal.bam -i "+SAMPLE+"N_recal.bam --algo TNscope --tumor_sample "+TUMOR+"T --normal_sample "+SAMPLE+"N --dbsnp "+data['dbsnp']+" --clip_by_minbq 1 --max_error_per_read 3 --min_init_tumor_lod 2.0 --min_base_qual 10 --min_base_qual_asm 10 --min_tumor_allele_frac 0.00005 "+TUMOR+"-tmpTNscope.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A1783T_recal.bam -i ABT414_FlankN_recal.bam --algo TNscope --tumor_sample A1783T --normal_sample ABT414_FlankN --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --clip_by_minbq 1 --max_error_per_read 3 --min_init_tumor_lod 2.0 --min_base_qual 10 --min_base_qual_asm 10 --min_tumor_allele_frac 0.00005 A1783-tmpTNscope.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i A9155T_recal.bam -i ABT414_FlankN_recal.bam --algo TNscope --tumor_sample A9155T --normal_sample ABT414_FlankN --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --clip_by_minbq 1 --max_error_per_read 3 --min_init_tumor_lod 2.0 --min_base_qual 10 --min_base_qual_asm 10 --min_tumor_allele_frac 0.00005 A9155-tmpTNscope.vcf.gz



### Tumor: A9878 Normal: ABT414_Flank

...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C0334T_recal.bam -i ABT414_FlankN_recal.bam --algo TNscope --tumor_sample C0334T --normal_sample ABT414_FlankN --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --clip_by_minbq 1 --max_error_per_read 3 --min_init_tumor_lod 2.0 --min_base_qual 10 --min_base_qual_asm 10 --min_tumor_allele_frac 0.00005 C0334-tmpTNscope.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C1572T_recal.bam -i ABT414_FlankN_recal.bam --algo TNscope --tumor_sample C1572T --normal_sample ABT414_FlankN --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --clip_by_minbq 1 --max_error_per_read 3 --min_init_tumor_lod 2.0 --min_base_qual 10 --min_base_qual_asm 10 --min_tumor_allele_frac 0.00005 C1572-tmpTNscope.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa -t 80 -i C8448T_recal.bam -i ABT414_FlankN_recal.bam --algo TNscope --tumor_sample C8448T --normal_sample ABT414_FlankN --dbsnp /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/variation/dbsnp-150.vcf.gz --clip_by_minbq 1 --max_error_per_read 3 --min_init_tumor_lod 2.0 --min_base_qual 10 --min_base_qual_asm 10 --min_tumor_allele_frac 0.00005 C8448-tmpTNscope.vcf.gz



In [113]:
display(Markdown("___"))
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command2 = data['SENTIEON_INSTALL_DIR']+"/bin/sentieon driver -t "+str(data['nt'])+" -r "+data['fasta']+" --algo TNModelApply --model "+data['ML_MODEL_T'] +" -v "+TUMOR+"-tmpTNscope.vcf.gz "+TUMOR+"-TNscope.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo TNModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonTNscopeModel_GiAB_HighAF_LowFP-201711.05.model -v A1783-tmpTNscope.vcf.gz A1783-TNscope.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo TNModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonTNscopeModel_GiAB_HighAF_LowFP-201711.05.model -v A9155-tmpTNscope.vcf.gz A9155-TNscope.vcf.gz



...

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo TNModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonTNscopeModel_GiAB_HighAF_LowFP-201711.05.model -v C0334-tmpTNscope.vcf.gz C0334-TNscope.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo TNModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonTNscopeModel_GiAB_HighAF_LowFP-201711.05.model -v C1572-tmpTNscope.vcf.gz C1572-TNscope.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/SENTIEON/sentieon-genomics-201911/bin/sentieon driver -t 80 -r /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa --algo TNModelApply --model /storage/gluster/vol1/data/PUBLIC/Tools/sentieon-genomics-201808.05/SentieonTNscopeModel_GiAB_HighAF_LowFP-201711.05.model -v C8448-tmpTNscope.vcf.gz C8448-TNscope.vcf.gz



In [114]:
display(Markdown("___"))
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command3 = data['bcfdir'] + " filter -s ML_FAIL -i \INFO/ML_PROB > 0.81 "+TUMOR+"-TNscope.vcf.gz -O z -m x -o "+TUMOR+ "-filtTNscope.vcf.gz"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

___

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i \INFO/ML_PROB > 0.81 A1783-TNscope.vcf.gz -O z -m x -o A1783-filtTNscope.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i \INFO/ML_PROB > 0.81 A9155-TNscope.vcf.gz -O z -m x -o A9155-filtTNscope.vcf.gz



...

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i \INFO/ML_PROB > 0.81 C0334-TNscope.vcf.gz -O z -m x -o C0334-filtTNscope.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i \INFO/ML_PROB > 0.81 C1572-TNscope.vcf.gz -O z -m x -o C1572-filtTNscope.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/SHARED/NGSTOOLS/BCBIO/bcbio/anaconda/bin/bcftools filter -s ML_FAIL -i \INFO/ML_PROB > 0.81 C8448-TNscope.vcf.gz -O z -m x -o C8448-filtTNscope.vcf.gz



# 9a. Somatic Variant Annotation

In [53]:
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command1 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+TUMOR+"-TNsnv.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+TUMOR+"-TNsnv.snpEff.vcf.gz"
    print(command1+'\n')
    if exec_scripts:
        subprocess.check_output(command1,shell=True)

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A1783-TNsnv.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A1783-TNsnv.snpEff.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A9155-TNsnv.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A9155-TNsnv.snpEff.vcf.gz



### Tumor: A9878 Normal: ABT414_Flank

...

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C0334-TNsnv.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C0334-TNsnv.snpEff.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C1572-TNsnv.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C1572-TNsnv.snpEff.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C8448-TNsnv.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C8448-TNsnv.snpEff.vcf.gz



In [115]:
display(Markdown("___"))
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command2 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+TUMOR+"-TNhaplotyper.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+TUMOR+"-TNhaplotyper.snpEff.vcf.gz"
    print(command2+'\n')
    if exec_scripts:
        subprocess.check_output(command2,shell=True)

___

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A1783-TNhaplotyper.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A1783-TNhaplotyper.snpEff.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A9155-TNhaplotyper.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A9155-TNhaplotyper.snpEff.vcf.gz



...

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C0334-TNhaplotyper.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C0334-TNhaplotyper.snpEff.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C1572-TNhaplotyper.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C1572-TNhaplotyper.snpEff.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C8448-TNhaplotyper.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C8448-TNhaplotyper.snpEff.vcf.gz



In [116]:
display(Markdown("___"))
for i in range(batchSize):
    SAMPLE = batch[i][0]
    TUMOR = batch[i][1]
    display(Markdown("### Tumor: "+TUMOR + " Normal: " + SAMPLE))
    command3 = "/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir="+data['SENTIEON_TMPDIR']+" eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 "+TUMOR+"-filtTNscope.vcf.gz | "+data['bgzipdir']+" --threads "+str(data['nt'])+" -c > "+TUMOR+"-filtTNscope.snpEff.vcf.gz"
    print(command3+'\n')
    if exec_scripts:
        subprocess.check_output(command3,shell=True)

___

### Tumor: A1783 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A1783-filtTNscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A1783-filtTNscope.snpEff.vcf.gz



### Tumor: A9155 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 A9155-filtTNscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > A9155-filtTNscope.snpEff.vcf.gz



...

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C0334-filtTNscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C0334-filtTNscope.snpEff.vcf.gz



### Tumor: C1572 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C1572-filtTNscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C1572-filtTNscope.snpEff.vcf.gz



### Tumor: C8448 Normal: ABT414_Flank

/storage/gluster/vol1/bcbio/anaconda/bin/snpEff -Xms1000m -Xmx36400m -Djava.io.tmpdir=/storage/gluster/vol1/SHARED/HOMEFOLDERS/adefalco/ABT414_Flank//$SAMPLE/tmpdir eff -noStats -t -noLog -dataDir /storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/snpeff -hgvs -noLof -i vcf -o vcf -noInteraction -noMotif -noNextProt -strict GRCh37.75 C8448-filtTNscope.vcf.gz | /glusterfs/biostorage1/brick1/data/PUBLIC/Tools/lib/bin/bgzip --threads 80 -c > C8448-filtTNscope.snpEff.vcf.gz



In [47]:
%%capture
#os.chdir(data['home'])
!rm html_pipeline.zip
!rm -r ./data
!mkdir data

In [48]:
%%capture
!jupyter nbconvert --no-input --execute --ExecutePreprocessor.timeout=-1 --output-dir='./data' --to html pipeline.ipynb
!cp /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/*.pdf ./data
!cp /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/*_N_aln_metrics.txt ./data
!cp /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/*_T_aln_metrics.txt ./data
!zip -r "html_pipeline" "./data"


In [ ]:
#!python -m nbconvert /home/adefalco/pipeline.ipynb  --to ipynb --execute

In [ ]:
#import nbinteract as nbi
#nbi.publish('AntonioDeFalco/testInteract/master', 'pipeline.ipynb')

In [ ]:
#!jupyter nbconvert /home/adefalco/pipeline.ipynb --to rst

In [ ]:
#!jupyter nbconvert /home/adefalco/pipeline.ipynb --to Interact